<a href="https://colab.research.google.com/github/TIMOTHYIS039/DataflowTemplates/blob/master/Langchain_Framework_OpenAI_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Link - https://colab.research.google.com/drive/1GxJEJwfgDpEMERi1GT2OFFMtxlHNfvZD?usp=sharing

## Install Necessary Packages

In [ ]:
!pip install langchain==0.0.164 openai tiktoken google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.0/789.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=2005c8898f4e58dda44ef08bfc4e3e83cf2000bf40b80f571a207334b1da20e8
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.2.1
    Uninstalling pydantic-2.2.1:
      Successfully uninstalled pydant

## Import Required Libraries

In [ ]:
import os
from langchain.llms import OpenAI,AzureOpenAI #if you are using Azure key, then use AzureOpenAI Class
from langchain.chat_models import ChatOpenAI,AzureChatOpenAI
from langchain import PromptTemplate, LLMChain, FewShotPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

## Load API Keys

In [ ]:
os.environ["OPENAI_API_TYPE"] = 'azure'
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = 'https://sandbox-ey.openai.azure.com/'
os.environ["OPENAI_API_KEY"] = '4699cbb6884a438095ba926bc0a8e12d'
os.environ['SERPAPI_API_KEY'] = "18e890ed0535f75ac185905d99dac7f4513907443f050acf527fa94b01f737a2"
davinci_model = 'text-davinci-003'
embedding_model = 'text-embedding-ada-002'
chat_model = 'gpt-35-turbo'

## Prompts In Langchain

**Prompts**

Language models take text as input - that text is commonly referred to as a prompt.

**Prompt Template**

A prompt template refers to a reproducible way to generate a prompt. It contains a text string (“the template”), that can take in a set of parameters from the end user and generate a prompt.

The prompt template may contain:

* instructions to the language model,
* a set of few shot examples to help the language model generate a better response,
* a question to the language model.

#### Single Input Prompt

In [ ]:
tagline_prompt = """Your task is generate some catchy and appealing tagline for company, delimited by triple `.
You should act as a marketing ads agent.
```{company}```"""

In [ ]:
#Class to make template for prompt.
tagline_prompt_template = PromptTemplate(
    input_variables=["company"], #Can pass any input variables dynamically
    template=tagline_prompt,
)
tagline_prompt_template.format(company="Ice-cream Shop") #can see how prompt will look like

'Your task is generate some catchy and appealing tagline for company, delimited by triple `.\nYou should act as a marketing ads agent.\n```Ice-cream Shop```'

In [ ]:
print(tagline_prompt_template.format(company="Book Shop")) #can see how prompt will look like

Your task is generate some catchy and appealing tagline for company, delimited by triple `.
You should act as a marketing ads agent.
```Book Shop```


#### Multiple Input Prompt

In [ ]:
qa_prompt = """Your task is to answer the asked question, delimited by triple `, from the context, delimited by #. Return the response in json format
Question - ```{question}```
Context - #{context}#"""

In [ ]:
#Class to make template for prompt.
qa_prompt_template = PromptTemplate(
    input_variables=["question","context"], #Can pass any input variables dynamically
    template=qa_prompt,
)
context = """According to a recent study published in a leading medical journal, lack of sleep can have a significant impact on mental health. The study found that people who regularly get less than 7 hours of sleep per night are more likely to experience depression, anxiety, and other mental health problems. Additionally, the study found that poor sleep quality can also contribute to these issues. The authors of the study recommend that individuals prioritize getting enough sleep as part of their overall self-care routine."""
question = "What are some common mental health issues that can be caused by a lack of sleep?"
qa_prompt_template.format(question=question,context=context) #can see how prompt will look like

'Your task is to answer the asked question, delimited by triple `, from the context, delimited by #. Return the response in json format\nQuestion - ```What are some common mental health issues that can be caused by a lack of sleep?```\nContext - #According to a recent study published in a leading medical journal, lack of sleep can have a significant impact on mental health. The study found that people who regularly get less than 7 hours of sleep per night are more likely to experience depression, anxiety, and other mental health problems. Additionally, the study found that poor sleep quality can also contribute to these issues. The authors of the study recommend that individuals prioritize getting enough sleep as part of their overall self-care routine.#'

#### Serialize your prompt

In [ ]:
#tagline_prompt_template.save('tagline_prompt.json')

#### Pass few shot examples to a prompt template

In [ ]:
few_shots = [
    {"review": "This is awesome!", "class_": "+"},
    {"review": "This is bad!", "class_": "-"},
    {"review": "Wow that movie was amazing", "class_": "+"},
]

In [ ]:
fewshots_prompt = """
Review: {review}
Class: {class_}\n
"""

In [ ]:
fewshots_prompt_template = PromptTemplate(
    input_variables=["review", "class_"],
    template=fewshots_prompt,
)

In [ ]:
fewshots_prompt_template.format(review='This is awesome',class_='+')

'\nReview: This is awesome\nClass: +\n\n'

In [ ]:
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=few_shots,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=fewshots_prompt_template,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Assign Class to given Review",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Review: {review}\nClass:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["review"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)

In [ ]:
print(few_shot_prompt.format(review="The movie was damn awesome"))

Assign Class to given Review


Review: This is awesome!
Class: +




Review: This is bad!
Class: -




Review: Wow that movie was amazing
Class: +



Review: The movie was damn awesome
Class:


## Access Models in Langchain

* **LLMs**

Large Language Models (LLMs) are the first type of models we cover. These models take a text string as input, and return a text string as output.

* **Chat Models**

Chat Models are the second type of models we cover. These models are usually backed by a language model, but their APIs are more structured. Specifically, these models take a list of Chat Messages as input, and return a Chat Message.

There are 3 main roles:
* User => end user, you
* Assistant => ChatGPT, model
* System => Acts as a whisperer to assistant’s ear and guiding its responses without the user being aware of it

* **Text Embedding Models**

The third type of models we cover are text embedding models. These models take text as input and return a list of floats.

### LLM in Langchain

In [ ]:
llm = AzureOpenAI(deployment_name=davinci_model,model_name="text-davinci-003", n=2, best_of=2)

#### Make a single query

In [ ]:
llm("write a poem on python programming?")

"\n\nProgramming with Python is an easy ride\nIt's powerful and yet simple to apply\nIt's really nice for coding and debugging\nA fun language to be exploring\n\nWe can use it for automation\nData analysis and visualization\nIt's also great for scientific computing\nNo need for tedious computing\n\nPython's great for web development\nAnd scripting tasks, just a breeze\nIt supports object-oriented code\nFor applications, it's the key\n\nWith Python, we can build websites\nAnd create powerful algorithms\nIt's an invaluable language\nFor software development, it's the norm\n\nPython is an amazing language\nIt's here to stay, no doubt\nIts remarkable features will keep us\nIn programming, we won't be left out"

#### Make multiple inputs to the model

In [ ]:
output = llm.generate(["write a poem on python programming?","What is the capital of India?"])

In [ ]:
output.generations

[[Generation(text="\n\nProgramming in Python is a grand old game \nIt's useful and helpful, though sometimes a pain \nIt can be daunting, but keep up the pace \nAnd you'll soon be an expert in this grand language of grace \n\nIt'll take time and practice, no doubt about that \nBut soon you'll be writing code that's top-notch and fat \nHere's to mastering Python, and pushing the limit \nIt's a language that's helpful, no matter how you spin it \n\nFrom web development to data science, Python's the way\nIt's a powerful language, here to stay \nSo keep practicing, and you'll be just fine \nPython's here to stay, so make it your lifeline!", generation_info={'finish_reason': 'stop', 'logprobs': None}),
  Generation(text="\n\nPython programming is a great way\nTo learn the basics of coding each day\nIt's efficient, intuitive, and powerful too\nSo many things you can do\n\nStart with the basics of syntax and loops\nThen move onto working with objects and groups\nOnce you've mastered the funda

#### Check the number of tokens in Prompt

In [ ]:
prompt = """We present IMAGEBIND, an approach to learn a joint embedding across six different modalities - images, text, audio, depth, thermal, and IMU data. We show that all combinations of paired data are not necessary to train such a joint
embedding, and only image-paired data is sufficient to bind the modalities together. IMAGEBIND can leverage recent large scale vision-language models, and extends their zeroshot capabilities to new modalities just by using their natural pairing with images. It enables novel emergent applications ‘out-of-the-box’ including cross-modal retrieval, composing modalities with arithmetic, cross-modal detection
and generation. The emergent capabilities improve with the strength of the image encoder and we set a new state-of-theart on emergent zero-shot recognition tasks across modalities, outperforming specialist supervised models. Finally,
we show strong few-shot recognition results outperforming prior work, and that IMAGEBIND serves as a new way to evaluate vision models for visual and non-visual tasks."""
words_len = len(prompt.split(' '))
token_len = llm.get_num_tokens(prompt)
print(f'Number of words and tokens - {words_len} and {token_len}')

Number of words and tokens - 144 and 227


### ChatModel in Langchain

In [ ]:
chat = AzureChatOpenAI(temperature = 0,deployment_name=chat_model,model_name=chat_model)

#### Basic Prompt for Chat Model

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

#### Prompt with Instructions for ChatModel

In [ ]:
template="You are a helpful assistant that translates english to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])

In [ ]:
print(chat_prompt.format(text="I love programming."))

System: You are a helpful assistant that translates english to pirate.
Human: Hi
AI: Argh me mateys
Human: I love programming.


### Embedding Model in Langchain

In [ ]:
embeddings = OpenAIEmbeddings(deployment=embedding_model)

In [ ]:
sentence = "I need to craete embedding of this sentece"

In [ ]:
embd = embeddings.embed_query(sentence)
len(embd)

1536

## Chains in Langchain

Using an LLM in isolation is fine for some simple applications, but many more complex ones require chaining LLMs - either with each other or with other experts. LangChain provides a standard interface for Chains, as well as some common implementations of chains for ease of use.

Chains are what you get by connecting one or more large language models (LLMs) in a logical way. (Chains can be built of entities other than LLMs but for now, let’s stick with this definition for simplicity).

#### Run a chain in Langchain

##### Using Non-Chat Model

In [ ]:
prompt_tagline = PromptTemplate(
    input_variables=["product"],
    template="Write a catchy and appealing tagline for {product}?",
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_tagline)

In [ ]:
print(chain.run("Ice-cream shop"))

In [ ]:
print(chain.run("Book shop"))



"Discover Your Next Adventure - Shop Books Now!"


##### Using Chat Model

In [ ]:
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="Write a catchy and appealing tagline for {product}?",
            input_variables=["product"],
        )
    )

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
chat_chain = LLMChain(llm=chat, prompt=chat_prompt_template)

In [ ]:
chat_chain.run({'product':"Ice-cream shop"})

'"Scoops of happiness in every cone!"'

#### Multiple values in a single Chain

In [ ]:
qa_prompt = """Your task is to answer the asked question, delimited by triple `, from the context, delimited by #. Return the response in json format
Question - ```{question}```
Context - #{context}#"""
#Class to make template for prompt.
qa_prompt_template = PromptTemplate(
    input_variables=["question","context"], #Can pass any input variables dynamically
    template=qa_prompt,
)
context = """According to a recent study published in a leading medical journal, lack of sleep can have a significant impact on mental health. The study found that people who regularly get less than 7 hours of sleep per night are more likely to experience depression, anxiety, and other mental health problems. Additionally, the study found that poor sleep quality can also contribute to these issues. The authors of the study recommend that individuals prioritize getting enough sleep as part of their overall self-care routine."""
question = "What are some common mental health issues that can be caused by a lack of sleep?"

In [ ]:
chain = LLMChain(llm=llm, prompt=qa_prompt_template)

In [ ]:
print(chain.run({'question':question,'context':context}))



{
  "answer": "The study found that people who regularly get less than 7 hours of sleep per night are more likely to experience depression, anxiety, and other mental health problems. Additionally, the study found that poor sleep quality can also contribute to these issues."
}


#### Debug a Chain (verbose=True)

In [ ]:
debug_chain = LLMChain(llm=llm, prompt=qa_prompt_template,verbose=True)

In [ ]:
print(debug_chain.run({'question':question,'context':context}))



> Entering new LLMChain chain...
Prompt after formatting:
Your task is to answer the asked question, delimited by triple `, from the context, delimited by #. Return the response in json format
Question - ```What are some common mental health issues that can be caused by a lack of sleep?```
Context - #According to a recent study published in a leading medical journal, lack of sleep can have a significant impact on mental health. The study found that people who regularly get less than 7 hours of sleep per night are more likely to experience depression, anxiety, and other mental health problems. Additionally, the study found that poor sleep quality can also contribute to these issues. The authors of the study recommend that individuals prioritize getting enough sleep as part of their overall self-care routine.#

> Finished chain.


{
  "response": "Depression, anxiety, and other mental health problems are some of the most common mental health issues that can be caused by a lack of sleep

#### Running Series of Chains

In [ ]:
# first prompt to get the company name
first_prompt = prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

# second prompt to get catchphrase for the company name
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a tagline for the following company: {company_name}",
)

In [ ]:
first_chain = LLMChain(llm=llm, prompt=first_prompt)
second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
complete_chain = SimpleSequentialChain(chains=[first_chain, second_chain], verbose=True)

In [ ]:
catchphrase = complete_chain.run("Delicious Pizza")
print(catchphrase)



> Entering new SimpleSequentialChain chain...


Pizza Paradise.


"A Slice of Heaven in Every Bite!"

> Finished chain.


"A Slice of Heaven in Every Bite!"


## Agent in Langchain (Optional)

**Tools**

Tools are ways that an agent can use to interact with the outside world.

**Agents**

Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user’s input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

**Agent Executor**

Agent executors take an agent and tools and use the agent to decide which tools to call and in what order.

In this part of the documentation we cover other related functionality to agent executors

### Loading Tools

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("what is the market capital of Apple?")



> Entering new AgentExecutor chain...
 I need to access information about the stock market
Action: Search
Action Input: "Apple Market Capital"
Observation: Apple Inc. is an American multinational technology company headquartered in Cupertino, California. Apple is the world's largest technology company by revenue, with US$394.3 billion in 2022 revenue. As of March 2023, Apple is the world's biggest company by market capitalization.
Thought: I now know the final answer
Final Answer: As of March 2023, Apple's market capitalization was US$394.3 billion.

> Finished chain.


"As of March 2023, Apple's market capitalization was US$394.3 billion."

### Calling using LLM only

In [ ]:
llm("what is 2 raised to the power .43?")

'\n\n2^.43 = 1.5229'

### Calling using LLM Calculation Chain

In [ ]:
agent.run("what is 2 raised to the power .43?")



> Entering new AgentExecutor chain...
 I need to calculate this
Action: Calculator
Action Input: 2^.43
Observation: Answer: 1.3472335768656902
Thought: I now know the final answer
Final Answer: 2^.43 = 1.3472335768656902

> Finished chain.


'2^.43 = 1.3472335768656902'